In [1]:
import sqlite3
import pandas as pd

# DB 생성 및 연결

In [76]:
# SQLite 데이터베이스 파일 생성
conn = sqlite3.connect('ASAC_MAP.db')
cursor = conn.cursor()

In [197]:
# csv to sqlite 변환 함수

def csv_to_sqlite(csv_file, db_file, table_name, chunk_size=1000):
    try:
        # SQLite 데이터베이스 파일 연결
        conn = sqlite3.connect(db_file)
        
        # CSV 파일을 청크 단위로 읽어 SQLite로 변환
        for chunk in pd.read_csv(csv_file, chunksize=chunk_size):  # 여기에서 괄호 ()를 반드시 사용해야 합니다.
            chunk.to_sql(table_name, conn, if_exists='append', index=False)
        
        # 변경 사항 저장
        conn.commit()
        print(f"CSV 파일을 SQLite 데이터베이스 '{table_name}' 테이블로 변환 완료")
    
    except Exception as e:
        print(f"오류 발생: {e}")
    
    finally:
        # 연결 종료
        conn.close()


# # 함수 사용 예시
# csv_to_sqlite('your_data.csv', 'mydatabase.db', 'review')

## Review table 

In [78]:
review_csv = f'csv/review.csv'

In [79]:

# CSV 파일을 Pandas DataFrame으로 읽어들입니다.
df = pd.read_csv(review_csv)

# DataFrame의 상위 5개 행을 출력합니다.
print(df.head())


   review_id  user_id  business_id  stars  \
0    1443659   607521       108373    5.0   
1    1044648   182108       134961    5.0   
2    5693786   749220        54049    5.0   
3    1166589   572170         9263    2.0   
4    6678286   816928       108373    5.0   

                                                text                 date  \
0  Found 51st Deli on Uber Eats and we loved it!!...  2017-01-14 21:23:10   
1  Love love love this place for HH n dinner is g...  2017-08-07 23:48:51   
2  Not sure the significance behind the name and ...  2018-09-25 14:49:55   
3  The service is going to be slooooww. And the f...  2017-10-02 01:52:47   
4  Much more than a deli! Wife and I had the Phil...  2017-12-29 20:18:59   

   food  service  atmosphere  facility  price  others  
0     0        1           0         0      0       1  
1     1        0           0         1      0       1  
2     1        0           0         1      0       1  
3     1        1           0         0    

In [54]:
# reveiw table 추가
csv_to_sqlite(review_csv, 'ASAC_MAP.db', 'review')

CSV 파일을 SQLite 데이터베이스 'review' 테이블로 변환 완료


## Business table

In [226]:
business = pd.read_csv('csv/business_raw.csv')

In [227]:
business.head()

,business_id,name,address,category,region,review_count_biz,average_stars_biz,service_score,others_score,food_score,price_score,atmosphere_score,facility_score,atmosphere_keyword,facility_keyword,food_keyword,others_keyword,price_keyword,service_keyword
0,110650,M Restaurant,231 S 8th St,"['Food', 'Desserts']","Philadelphia, 19106",51.0,3.705882,0.086116,0.235483,0.235545,0.206473,0.464060,-0.060803,outdoor dinner,sat inside,ordered chicken,bad restaurant,drink prices,restaurant presented
1,110907,Baker Boss Cafe,5694 Nolensville Pike,"['Restaurants', 'Vietnamese']","Nashville, 37211",59.0,4.406780,0.363836,0.348599,0.490465,0.589583,0.437653,0.238883,nolsenville place,trove restaurants,snack vietnamese,vietnamese restaurant,pricing menu,restaurant complaint
2,28638,Embassy Suites by Hilton Tampa Downtown Conven...,513 S Florida Ave,"['Restaurants', 'Cafes']","Tampa, 33602",108.0,2.212963,-0.240082,-0.081164,0.182467,-0.359529,0.307114,-0.165903,gasparilla hotel,rooms cleaned,stay tampa,hilton hotels,hotel cost,cancellation hotel
3,100086,Chili's,1741 Manhattan Blvd,"['Restaurants', 'American (Traditional)']","Harvey, 70058",88.0,1.863636,-0.268126,-0.212355,-0.217972,-0.179063,-0.177152,-0.468663,restaurant,restaurant bathrooms,steak expected,restaurant experience,military discount,waitress order
4,136473,The Blind Tiger Cafe - Westchase,10118 Montague St,"['Restaurants', 'Sandwiches']","Tampa, 33626",99.0,3.787879,0.264892,0.356398,0.438167,-0.019682,0.522439,0.387926,tiger cafe,westchase layout,coffee options,westchase coffee,drinks overpriced,latte avocado


In [228]:
business.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 651 entries, 0 to 650
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   business_id         651 non-null    int64  
 1   name                651 non-null    object 
 2   address             644 non-null    object 
 3   category            651 non-null    object 
 4   region              651 non-null    object 
 5   review_count_biz    651 non-null    float64
 6   average_stars_biz   651 non-null    float64
 7   service_score       648 non-null    float64
 8   others_score        649 non-null    float64
 9   food_score          648 non-null    float64
 10  price_score         510 non-null    float64
 11  atmosphere_score    599 non-null    float64
 12  facility_score      593 non-null    float64
 13  atmosphere_keyword  600 non-null    object 
 14  facility_keyword    593 non-null    object 
 15  food_keyword        650 non-null    object 
 16  others_k

In [229]:
# 결측치 처리
business['facility_score'] = business['facility_score'].fillna(0)
business['service_score'] = business['service_score'].fillna(0)
business['others_score'] = business['others_score'].fillna(0)
business['food_score'] = business['food_score'].fillna(0)
business['price_score'] = business['price_score'].fillna(0)
business['atmosphere_score'] = business['atmosphere_score'].fillna(0)


business['food_keyword']= business['food_keyword'].fillna('none')
business['others_keyword'] = business['others_keyword'].fillna('none')
business['atmosphere_keyword'] = business['atmosphere_keyword'].fillna('none')
business['facility_keyword'] = business['facility_keyword'].fillna('none')
business['service_keyword'] = business['service_keyword'].fillna('none')
business['price_keyword'] = business['price_keyword'].fillna('none')


In [233]:
business.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 651 entries, 0 to 650
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   business_id         651 non-null    int64  
 1   name                651 non-null    object 
 2   address             644 non-null    object 
 3   category            651 non-null    object 
 4   region              651 non-null    object 
 5   review_count_biz    651 non-null    float64
 6   average_stars_biz   651 non-null    float64
 7   service_score       651 non-null    float64
 8   others_score        651 non-null    float64
 9   food_score          651 non-null    float64
 10  price_score         651 non-null    float64
 11  atmosphere_score    651 non-null    float64
 12  facility_score      651 non-null    float64
 13  atmosphere_keyword  651 non-null    object 
 14  facility_keyword    651 non-null    object 
 15  food_keyword        651 non-null    object 
 16  others_k

In [241]:
#DF to sql

def df_to_sqlite(df, db_file, table_name, if_exists='append'):
    try:
        # SQLite 데이터베이스 파일 연결
        conn = sqlite3.connect(db_file)
        
        # 데이터프레임을 SQL 테이블로 저장
        df.to_sql(table_name, conn, if_exists=if_exists, index=False)
        
        # 변경 사항 저장
        conn.commit()
        print(f"데이터프레임을 SQLite 데이터베이스 '{table_name}' 테이블로 저장 완료")
    
    except Exception as e:
        print(f"오류 발생: {e}")
    
    finally:
        # 연결 종료
        conn.close()

# 데이터프레임을 데이터베이스에 저장
db_file_path = 'ASAC_MAP.db'
table_name = 'business'
df_to_sqlite(business, db_file_path, table_name, if_exists='append')  # 'replace', 'append' 또는 'fail'


데이터프레임을 SQLite 데이터베이스 'business' 테이블로 저장 완료


## User table

In [59]:
user_csv = f'csv/user.csv'

In [60]:
user_df=pd.read_csv('csv/user.csv')
user_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27012 entries, 0 to 27011
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   user_id              27012 non-null  int64  
 1   name                 27012 non-null  object 
 2   review_count_user    27012 non-null  float64
 3   average_stars_user   27012 non-null  float64
 4   most_visited_region  27012 non-null  object 
 5   visit_cnt            27012 non-null  float64
 6   log_visit_cnt        27012 non-null  float64
dtypes: float64(4), int64(1), object(2)
memory usage: 1.4+ MB


In [61]:
csv_to_sqlite(user_csv, 'ASAC_MAP.db', 'user')

CSV 파일을 SQLite 데이터베이스 'user' 테이블로 변환 완료


## table 조회

In [242]:
import sqlite3

# 데이터베이스 파일에 연결
conn = sqlite3.connect('ASAC_MAP.db')
cursor = conn.cursor()

# 테이블 조회 쿼리 실행
cursor.execute("SELECT COUNT(*) FROM business")

# 결과를 가져옴
rows = cursor.fetchall()

# 결과 출력
for row in rows:
    print(row)



(651,)


In [120]:
import sqlite3

# 데이터베이스 파일에 연결
conn = sqlite3.connect('ASAC_MAP.db')
cursor = conn.cursor()

# 테이블 조회 쿼리 실행
cursor.execute("SELECT count(*) FROM review")

# 결과를 가져옴
rows = cursor.fetchall()

# 결과 출력
for row in rows:
    print(row)



(30879,)


In [88]:
import sqlite3

# 데이터베이스 파일에 연결
conn = sqlite3.connect('ASAC_MAP.db')
cursor = conn.cursor()

# 테이블 조회 쿼리 실행
cursor.execute("SELECT count(*) FROM user")

# 결과를 가져옴
rows = cursor.fetchall()

# 결과 출력
for row in rows:
    print(row)



(27012,)


In [225]:
# 모든 테이블 이름 조회
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()


# 테이블 이름 출력
for table in tables:
    print(table[0])


review
user


In [91]:
import sqlite3

# 데이터베이스 파일 경로 지정
database_path = 'ASAC_MAP.db'

# 데이터베이스 연결
conn = sqlite3.connect(database_path)


# 쿼리 실행
biz_id = 22138
query = "SELECT * FROM review WHERE business_id = ?;"
df = pd.read_sql_query(query, conn, params=(biz_id,))
# 결과 출력
print(df)

# 데이터베이스 연결 종료
conn.close()


     review_id  user_id  business_id  stars  \
0      3350125   262704        22138    4.0   
1      6231996   565964        22138    4.0   
2      6823760  1479080        22138    5.0   
3      6421669  1493874        22138    5.0   
4      3249470  1704373        22138    5.0   
..         ...      ...          ...    ...   
138    2995022   103858        22138    5.0   
139    6136815   571940        22138    5.0   
140    4499170   980619        22138    5.0   
141     108011  1578922        22138    5.0   
142    1847631   284928        22138    5.0   

                                                  text                 date  \
0    Really good coffee and not busy. Good quiet pl...  2018-03-23 13:43:28   
1    The iced coffee had a nice kick to it & I will...  2018-01-21 15:46:24   
2    Love this place! I don't work in Clayton anymo...  2018-04-30 06:31:58   
3    The atmosphere is a little cold but the staff ...  2018-06-20 14:28:49   
4    Oh my!  I'm having my first "Pour O

In [92]:
# 연결 종료
conn.close()


테이블 삭제


In [238]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('ASAC_MAP.db')
cursor = conn.cursor()

# SQL command to drop the review table
drop_table_command = "DROP TABLE IF EXISTS business;"

try:
    cursor.execute(drop_table_command)
    conn.commit()  # Commit the changes to the database
    print(" table has been deleted successfully.")
except sqlite3.Error as e:
    print(f"An error occurred: {e}")

# finally:
#     # Close the cursor and connection to SQLite
#     cursor.close()
#     conn.close()


 table has been deleted successfully.
